In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import re
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
#url_list = [f'https://www.avto.net/Ads/results.asp?znamka=&model=&modelID=&tip=katerikoli%20tip&znamka2=&model2=&tip2=katerikoli%20tip&znamka3=&model3=&tip3=katerikoli%20tip&cenaMin=0&cenaMax=999999&letnikMin=2000&letnikMax=2090&bencin=201&starost2=999&oblika=0&ccmMin=0&ccmMax=99999&mocMin=&mocMax=&kmMin=0&kmMax=9999999&kwMin=0&kwMax=999&motortakt=&motorvalji=&lokacija=0&sirina=&dolzina=&dolzinaMIN=&dolzinaMAX=&nosilnostMIN=&nosilnostMAX=&lezisc=&presek=&premer=&col=&vijakov=&EToznaka=&vozilo=&airbag=&barva=&barvaint=&EQ1=1000000000&EQ2=1000000000&EQ3=1000000000&EQ4=100000000&EQ5=1000000000&EQ6=1000000000&EQ7=1000100120&EQ8=1010000001&EQ9=100000000&KAT=1010000000&PIA=&PIAzero=&PSLO=&akcija=&paketgarancije=&broker=&prikazkategorije=&kategorija=&zaloga=&arhiv=&presort=&tipsort=&stran={i}' for i in range(1,22)]
url_list = [f'https://www.avto.net/Ads/results.asp?znamka=Citroen&model=Xsara%20Picasso&modelID=&tip=katerikoli%20tip&znamka2=&model2=&tip2=katerikoli%20tip&znamka3=&model3=&tip3=katerikoli%20tip&cenaMin=0&cenaMax=999999&letnikMin=2000&letnikMax=2090&bencin=202&starost2=999&oblika=0&ccmMin=0&ccmMax=99999&mocMin=&mocMax=&kmMin=0&kmMax=9999999&kwMin=0&kwMax=999&motortakt=&motorvalji=&lokacija=0&sirina=&dolzina=&dolzinaMIN=&dolzinaMAX=&nosilnostMIN=&nosilnostMAX=&lezisc=&presek=&premer=&col=&vijakov=&EToznaka=&vozilo=&airbag=&barva=&barvaint=&EQ1=1000000000&EQ2=1000000000&EQ3=1000000000&EQ4=100000000&EQ5=1000000000&EQ6=1000000000&EQ7=1000100120&EQ8=1010000001&EQ9=100000000&KAT=1010000000&PIA=&PIAzero=&PSLO=&akcija=&paketgarancije=&broker=&prikazkategorije=&kategorija=&zaloga=&arhiv=&presort=&tipsort=&stran={i}' for i in range(1,22)]

In [3]:
headers = {
    'User-Agent': '*',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

s = requests.Session()
s.headers.update(headers)

request_list = [s.get(url_list[i],allow_redirects=False) for i in range(len(url_list))]
request_list = np.hstack(request_list)

In [8]:
ci = []
cif = []

for r in request_list:
    soup = BeautifulSoup(r.text, 'html.parser')
    car_list = [
        x for x in soup.find_all('div') if str(x).startswith('<div class="Res') and 
        'FOTO' in str(x) and 
        'DATA' in str(x) and
        'CENA' in str(x)            
    ]
    for car in car_list:
        try:
            url = 'https://www.avto.net/' + re.findall(r'a class="Adlink" href=\"([\S  ]*)\"', str(car))[0][3:]
            name = re.findall(f'title=\"([\S  ]*)\"/>', str(car))[0]
            year, mileage, volume, power, torque = [int(x) for x in re.findall(f'<li>Letnik 1.registracije:([\w]*)</li>\n<li>([\w]*) km</li><li>diesel motor, ([\w]*) ccm, ([\w]*) kW  / ([\w]*) KM</li>', str(car))[0]]
            price = int(re.findall(f'EUR=([\w]*)', str(car))[0])
            ci.append([name, year, mileage, volume, power, torque, price, url])
        except:
            cif.append(car)
            
ci = np.array(ci)
cif = np.array(cif)

In [13]:
train, test = train_test_split(ci, shuffle=True, test_size=0.2)

In [14]:
import lightgbm as lgbm
lgbm_classifier = lgbm.LGBMRegressor()


distributions = dict(
    num_leaves = [5,10,20,30,50],
    max_depth = [-1,1,2,3,4,5,6],
    learning_rate = [0.005, 0.001, 0.05, 0.01, 0.1, 0.5],
    n_estimators = [20,30,50,100,200,300,500]
)


clf = GridSearchCV(lgbm_classifier, distributions)
search = clf.fit(train[:,1:-2].astype(int), train[:,-2].astype(int))
search.best_params_

{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 200, 'num_leaves': 5}

In [18]:
model = search.best_estimator_

In [19]:
model.predict([[2003,220000,2000,66,90]])

array([1249.04650523])

In [45]:
mask  = (ci[:,1].astype(int) == 2015)
mask &= (ci[:,2].astype(int) > 40000) & (ci[:,2].astype(int) < 70000)
mask &= (ci[:,3].astype(int) > 1000) & (ci[:,3].astype(int) < 2000)
mask &= (ci[:,-2].astype(int) > 6000) & (ci[:,-2].astype(int) < 9000)

for info in ci[mask]:
    print(info[-2], info[-1])

7200 https://www.avto.net/Ads/details.asp?id=14932990&amp;display=Fiat 500 1.2 8V Sport
6850 https://www.avto.net/Ads/details.asp?id=15062427&amp;display=ÂŠkoda Rapid Spaceback 1.2 TSI Active Red...
7350 https://www.avto.net/Ads/details.asp?id=15061878&amp;display=Seat Ibiza ST 1.4 16V Jubilee Style
7990 https://www.avto.net/Ads/details.asp?id=15061550&amp;display=Renault Clio 1.2 16V Techno Feel-SLOVENS...
8290 https://www.avto.net/Ads/details.asp?id=15059857&amp;display=Kia Rio 1.2 EX FUN
8800 https://www.avto.net/Ads/details.asp?id=15059060&amp;display=Mazda Mazda2 G90 Challenge
7599 https://www.avto.net/Ads/details.asp?id=15059054&amp;display=Renault Clio 1.2 16V Techno Feel SLO RED...
8990 https://www.avto.net/Ads/details.asp?id=15058928&amp;display=Kia Pro_CeeD 1.4 CVVT LX Fun+
6490 https://www.avto.net/Ads/details.asp?id=15058830&amp;display=Peugeot 208 1.2 VTi PureTech Access
